# EDA 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [60]:
#read data
dl_during_request_df = pd.read_csv('../data/driver_locations_during_request.csv')
trip_df = pd.read_csv('../data/nb.csv')

In [61]:
dl_during_request_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1557740 entries, 0 to 1557739
Data columns (total 8 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   id             1557740 non-null  int64  
 1   order_id       1557740 non-null  int64  
 2   driver_id      1557740 non-null  int64  
 3   driver_action  1557740 non-null  object 
 4   lat            1557740 non-null  float64
 5   lng            1557740 non-null  float64
 6   created_at     0 non-null        float64
 7   updated_at     0 non-null        float64
dtypes: float64(4), int64(3), object(1)
memory usage: 95.1+ MB


In [62]:
trip_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536020 entries, 0 to 536019
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Trip ID           536020 non-null  int64 
 1   Trip Origin       536020 non-null  object
 2   Trip Destination  536020 non-null  object
 3   Trip Start Time   534369 non-null  object
 4   Trip End Time     536019 non-null  object
dtypes: int64(1), object(4)
memory usage: 20.4+ MB


### Checking missing values

In [63]:
dl_during_request_df.isnull().sum()

id                     0
order_id               0
driver_id              0
driver_action          0
lat                    0
lng                    0
created_at       1557740
updated_at       1557740
dtype: int64

As we can see there are 1557740 missing values in created_at and updated_at, since the dataset row count is 1557740. it is obvious that the two columns are null for all the rows. Therefore we will remove/drop them from the dataset.

In [75]:
trip_df.isnull().sum()

trip_id                0
trip_origin            0
trip_destination       0
trip_start_time     1651
trip_end_time          1
dtype: int64

As we can see we have 1651 missing values in the ``trip_start_time`` column and 1 at the ``trip_end_time`` we can assume that entries with null trip_start_time, are the once that got canceled after being accepted by the drivers.  

## Data Cleaning

In [65]:
#rename the column names of trip_df
trip_df = trip_df.rename(
    columns={'Trip ID': 'trip_id', 
    'Trip Origin': 'trip_origin', 
    'Trip Destination': 'trip_destination', 
    'Trip Start Time': 'trip_start_time', 
    'Trip End Time': 'trip_end_time',})
trip_df.columns

Index(['trip_id', 'trip_origin', 'trip_destination', 'trip_start_time',
       'trip_end_time'],
      dtype='object')

In [66]:
#drop created_at and updated_at columns
dl_during_request_df.drop(columns=['created_at', 'updated_at'], inplace=True)
dl_during_request_df.isnull().sum()

id               0
order_id         0
driver_id        0
driver_action    0
lat              0
lng              0
dtype: int64

In [83]:
#store rows in trip_df whose trip_start_time is null to a diffrent datafarme
cancelled_orders_df = trip_df[trip_df['trip_start_time'].isnull()]
cancelled_orders_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1651 entries, 665 to 535758
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   trip_id           1651 non-null   int64 
 1   trip_origin       1651 non-null   object
 2   trip_destination  1651 non-null   object
 3   trip_start_time   0 non-null      object
 4   trip_end_time     1651 non-null   object
dtypes: int64(1), object(4)
memory usage: 77.4+ KB


In [84]:
#now drop null rows
trip_df.dropna(inplace=True)
trip_df.isna().sum()

trip_id             0
trip_origin         0
trip_destination    0
trip_start_time     0
trip_end_time       0
dtype: int64

In [19]:
#accepted orders 
accepted_orders_df = dl_during_request_df[dl_during_request_df['driver_action'] == "accepted"]
accepted_orders_df.shape

(25903, 8)

## Merge The Two Datasets

In [49]:
merged_df = pd.merge(trip_df, accepted_orders_df[['order_id','driver_id', 'driver_action', 'lat', 'lng']], left_on='trip_id', right_on='order_id', how='right' )
merged_df.isnull().sum()

trip_id              0
trip_origin          0
trip_destination     0
trip_start_time     32
trip_end_time        0
order_id             0
driver_id            0
driver_action        0
lat                  0
lng                  0
dtype: int64


- nb.csv(i.e. trip_df) contains 535k orders created in the span of 6 months from July 2021 - December 2021
driver_locations_during_request.csv (i.e. dl_during_request_df) contains driver order requests and the location of the driver during the request for each order in the nb.csv (Trip ID). Since this is a very data intensive task our client calculated it only for one month.

- ``driver_locations_during_request.csv#order_id == nb.csv#trip_id``
